In [55]:
import os
import dill
import ipywidgets as widgets
from aepsych.plotting import plot_strat
from aepsych.server import AEPsychServer
from IPython.display import display
from ipywidgets import HTML, AppLayout, Box, Button, Label, Layout, Text
from IPython.core.display import HTML as html

database_path = None
server = AEPsychServer()
strat = None
# if database_path != None:
#     strat = server.get_strat_from_replay()

#---------- Style ----------
#    background: #0091EA
#   background: linear-gradient(20deg, #315485, #00BE?73);
display(html("""
<style>
.jp-CodeCell {
    width:100% !important;
    background: whitesmoke; 
}
.jp-Notebook .jp-Cell {
    padding: 0 !important;
}

.jupyter-widgets.widget-tab {
    margin: 40px 30px;
}

body.jp-Notebook {
    margin: 0 !important;
    padding: 0 !important;
    height: 100vh !important;
    background: whitesmoke;
}
.widget-inline-hbox {
    margin: 0px;
}
.widget-output {
    padding: 25px !important;
    border-radius: 2px;
    height: 100%;
    width: 100%;
    background: white;
    width: -webkit-fill-available !important;
    margin: 10px 30px !important;
    border: 1px solid #bdbdb
}
.widget-vbox {
    justify-content: center;
    margin: 0 1%
}
[data-jp-theme-light='true'] .jp-RenderedImage img.jp-needs-light-background {
    width: 100%;
}

.jupyter-widgets.widget-tab {
    min-width: 390px;
}
html {
    background: #0091EA;
    height: 100%;
    margin: 0px; 
    padding: 0px; 

}

</style>"""))

display(HTML("<style> *{margin:0; padding:0;} html, body, .container{margin:0;!important padding:0;!important} .container { width:100% !important;}</style>"))
# <img src="./test.png" alt="logo" width="40px" height="15px">
header = HTML(
    """
<div class="nav" style=" background: #0091EA;margin: 0px;border:">
    
    <h1 style="
    text-align: center;
    font-family: helvetica;
    font-weight: 400;
    font-size: 20px;
    display: flex;
    padding: 15px 20px;
    margin: 0px;
    hieght: 100px; 
    jusify-content: flex-start;
    color: white">Interactive AEPsych</h1>
</div>
"""
)
display(header)

input_style = {"description_width": "initial"}
input_layout= Layout(margin="5px 0")
btn_style = Layout(margin="20px 10px")

btn_box_layout = Layout(display="flex", justify_content="flex-end")
file_output = widgets.Output(
    # layout={"border": "1px solid black", "margin": "10px", "overflow": "scroll", "height": "40px", "padding": "5px"}
)


# ---------- Inputs -----------


def on_value_change(change):
    file_output.clear_output()
    with file_output:
        change["new"]
#         print(change["new"])

        
def on_value_change_input(change):
    inputs_output.clear_output()
    with inputs_output:
        change["new"]


uploader = widgets.FileUpload(accept=".db", multiple=False)

input_zero = widgets.Dropdown(
    options=[("Yes Trial", "Detected Trial"), ("No Trial", "Undetected Trial")],
    value="Detected Trial",
    description="0:",
    style=input_style,
    layout=input_layout
)
input_one = widgets.Dropdown(
    options=[("No Trial", "Undetected Trial"), ("Yes Trial", "Detected Trial")],
    value="Undetected Trial",
    description="1:",
    style=input_style,
    layout=input_layout
)

target_level = widgets.BoundedFloatText(
    value=0.75, min=0, max=1, 
    step=0.1, description="target_level:", disabled=False, 
    style=input_style, layout=input_layout
)

cred_level = widgets.BoundedFloatText(
    value=0.95, min=0, max=1, step=0.1, 
    description="cred_level:", disabled=False, 
    style=input_style, layout=input_layout
)

x_axis = Text(
    value="Angle (degrees)", placeholder="x axis label",
    description="x_axis:", disabled=False, style=input_style,
    layout=input_layout
)

y_axis = Text(
    value="Detection Probability", placeholder="y axis label", 
    description="y_axis:", disabled=False, style=input_style,
    layout=input_layout
)

# Observes input changes
uploader.observe(on_value_change, names="value")
# display(uploader, file_output)

inputs_output = widgets.Output(
    layout={"border": "1px solid black", "padding": "20px", "margin": "20px", "min-height ": "100px"}
)
input_zero.observe(on_value_change_input, names="value")
input_one.observe(on_value_change_input, names="value")
target_level.observe(on_value_change_input, names="value")
cred_level.observe(on_value_change_input, names="value")
x_axis.observe(on_value_change_input, names="value")
y_axis.observe(on_value_change_input, names="value")
# ------------ Buttons ------------

button_upload = widgets.Button(
    description="Upload", disabled=False, button_style="success", tooltip="Click to Upload", layout=btn_style
)
button_submit = widgets.Button(
    description="Submit", disabled=False, button_style="success", tooltip="submit", layout=btn_style
)
button_reset = widgets.Button(
    description="Reset", disabled=False, button_style="info", tooltip="reset", layout=btn_style
)

# --------- Accordion -----------
accordion = widgets.Accordion(
    children=[
        widgets.VBox([input_zero, input_one]),
        widgets.VBox(
            [
                target_level,
                cred_level,
            ]
        ),
        widgets.VBox([x_axis, y_axis]),
    ]
)
accordion.set_title(0, "Outcome Labels")
accordion.set_title(1, "Parameters")
accordion.set_title(2, "Axis Labels")

# ------------ Tabs -------------
tab = widgets.Tab()

children = [
    widgets.VBox(
        [
            Label("Resume Session:"),
            uploader,
            widgets.Box([button_upload], layout=btn_box_layout),
            file_output,
        ]
    ),
    widgets.VBox(
        [
            accordion,
            widgets.Box([button_submit, button_reset], layout=btn_box_layout),
            Label("Plot output: "),
            inputs_output,
        ]
    ),
]
tab.children = children
tab.set_title(0, "Upload")
tab.set_title(1, "Plot")
display(tab)

# ------- functions ------------


def start_server():
    global strat
    global server
    with file_output:
        # hard coded db path will be replaced with `database_path`
        try:
            server = AEPsychServer(database_path=database_path)
            strat = server.get_strat_from_replay()
            # server = AEPsychServer(database_path="/var/svcscm/persistent/private-90d/databasesdata_collection_analysis_tutorial.db")
            print("Connected...")
            display_plot()
        except: 
            print("Unable to connect to server...")


def display_plot():
    global strat
    inputs_output.clear_output()
    if strat != None:
        plot_strat(
            strat,
            xlabel=x_axis.value,
            ylabel=y_axis.value,
            yes_label=input_one.value,
            no_label=input_zero.value,
            cred_level=cred_level.value,
            target_level=target_level.value,
        )
    else:
        print("Server has no experiment records!")


def upload():
    global server
    global strat
    global database_path
    with file_output:
        print("\n -----Now this is how your file looks like:----- \n")
        file_output.clear_output()
        if uploader.value == {}:
            print("No file uploaded")
        else:
            # Maybe add a function to check file format
            for file_name, file in uploader.value.items():
                current_path = os.getcwd()
                # internal_target_path = current_path + "/persistent/private-90d"
                external_target_path = current_path + "/databases"
                completeName = None 
                # if os.path.isdir(internal_target_path):
                #     completeName = os.path.join(internal_target_path, file_name)
                if os.path.isdir(external_target_path):
                    completeName = os.path.join(external_target_path, file_name)
                else:
                    print("File upload unsuccessful")

                with open(completeName, "wb") as f:
                    f.write(file["content"])
                    if completeName != None:
                        database_path = completeName
                    start_server()
                    f.close()

def submit():
    with inputs_output:
        display_plot()
        file_output.clear_output()


def reset():
    with inputs_output:
        inputs_output.clear_output()
        #         plotting options
        x_axis.value = "Angle (degrees)"
        y_axis.value = "Detection Probability"
        input_one.value = "Detected Trial"
        input_zero.value = "Undetected Trial"
        cred_level.value = 0.75
        target_level.value = 0.95
        file_output.clear_output()


# Onclick handler
def upload_clicked(b):
    upload()


def submit_clicked(b):
    submit()


def reset_clicked(b):
    reset()


button_upload.on_click(upload_clicked)
button_submit.on_click(submit_clicked)
button_reset.on_click(reset_clicked)

2022-07-28 16:11:35,679 [INFO   ] Found DB at ./databases/default.db, appending!


HTML(value='<style> *{margin:0; padding:0;} html, body, .container{margin:0;!important padding:0;!important} .…

HTML(value='\n<div class="nav" style=" background: #0091EA;margin: 0px;border:">\n    \n    <h1 style="\n    t…